In [145]:
from pulp import *
import pandas as pd
from typing import List, Tuple

In [168]:
alternatives = pd.read_csv("./data/UTA_milosz_alternatives.csv")
criteria = pd.read_csv("./data/UTA_milosz_criteria.csv")

In [169]:
alternatives

,name,experience,qualities,expectation
0,A,6,4,12.5
1,B,5,0,10.0
2,C,7,8,13.0
3,D,5,2,14.5
4,E,10,5,15.0
5,F,9,9,14.0
6,G,3,3,10.5
7,H,1,7,12.0
8,I,0,10,12.5
9,J,2,5,11.5


In [170]:
criteria

,criterion_name,criterion_type,alpha,beta,weight
0,experience,gain,0.0,10,5
1,qualities,gain,0.0,10,2
2,expectation,cost,NaN,0,3


In [171]:
criteria["weight"] /= sum(criteria["weight"])
criteria

,criterion_name,criterion_type,alpha,beta,weight
0,experience,gain,0.0,10,0.5
1,qualities,gain,0.0,10,0.2
2,expectation,cost,NaN,0,0.3


In [172]:

def fill_alpha_beta(alternatives: pd.DataFrame, criteria: pd.DataFrame) -> pd.DataFrame:
    """
    If any criteria don't have the values for the worst (alpha) or best (beta) performance,
    make them equal to the worst/best performances attained by the alternatives on this criterion
    """
    for idx, criterion in criteria.iterrows():
        if pd.isnull(criterion["alpha"]):
            if criterion["criterion_type"] == "gain":
                criterion["alpha"] = min(alternatives[criterion["criterion_name"]])
            if criterion["criterion_type"] == "cost":
                print(criterion["criterion_name"], max(alternatives[criterion["criterion_name"]]))
                criterion["alpha"] = max(alternatives[criterion["criterion_name"]])

        if pd.isnull(criterion["beta"]):
            if criterion["criterion_type"] == "gain":
                criterion["beta"] = max(alternatives[criterion["criterion_name"]])
            if criterion["criterion_type"] == "cost":
                criterion["beta"] = min(alternatives[criterion["criterion_name"]])
        criteria.loc[idx] = criterion

In [173]:
fill_alpha_beta(alternatives, criteria)
criteria

expectation 15.0


,criterion_name,criterion_type,alpha,beta,weight
0,experience,gain,0.0,10,0.5
1,qualities,gain,0.0,10,0.2
2,expectation,cost,15.0,0,0.3



Expected preferences from project1:
- Path of Exile > Last Epoch
- Divinity Original Sin 2 > Gears Tactics
- Portal 2 > Superliminal
- Slay the Spire > Rogue Legacy 2

Additional comparisions:
- Terraria > Elden Ring
- Slay the Spire > Nebuchadnezzar
- Elden Ring > Nebuchadnezzar
- Nebuchadnezzar > Terraria (to introduce inconsistency)


In [228]:
# TODO: just to indicate what I intend to work on next
def ordinal_regression_model(alternatives: pd.DataFrame, criteria: pd.DataFrame, comparisions: List[Tuple[str, str, str]], num_breakpoints: int = 0) -> LpProblem:
    alternative_names = alternatives.iloc[:, 0].tolist()
    model = LpProblem(name="Ordinal Regression", sense=LpMinimize)

    # definig the objective function - sum of under- and over-estimation errors
    over_error_variables = LpVariable.dicts("over", alternative_names, lowBound=0)
    under_error_variables = LpVariable.dicts("under", alternative_names, lowBound=0)
    model += lpSum([x for x in list(under_error_variables.values())+list(over_error_variables.values())])

    # pairwise comparisions
    # TODO: think how to implement strict inequalities in the model
    sense_dict = dict([(">", LpConstraintGE), ("=", LpConstraintEQ), ("<", LpConstraintLE)])
    coeffs = [1, -1, 1]
    for a, b, relation in comparisions:
        a_over = over_error_variables[a]
        a_under = under_error_variables[a]
        b_over = over_error_variables[b]
        b_under = under_error_variables[b]
        U_a = LpVariable(name=f"U({a})", lowBound=0, upBound=1)
        U_b = LpVariable(name=f"U({b})", lowBound=0, upBound=1)
        sense = sense_dict[relation]
        left = lpSum([c*x for c, x in zip(coeffs, [U_a, a_over, a_under])])
        right = lpSum([[c*x for c, x in zip(coeffs, [U_b, b_over, b_under])]])
        # TODO: model STRICT inequality
        # transformed to caompare to 0, because that's what PuLP wants
        model += LpConstraint(left-right, sense=sense, name=f"{a}{relation}{b}", rhs=0)

    # all worst performances must be 0
    # the sum of best performances must be equal 1
    beta_variables = []
    for _, criterion in criteria.iterrows():
        criterion_name = criterion["criterion_name"]
        u_a = LpVariable(name=f"u_a_{criterion_name}", lowBound=0, upBound=0)
        # we assume that weights are normalised already
        u_b = LpVariable(name=f"u_b_{criterion_name}", lowBound=criterion["weight"], upBound=criterion["weight"])
        beta_variables.append(u_b)
        model += (u_a == 0, f"u_a_{criterion_name}")
    model += LpConstraint(lpSum(beta_variables), sense=LpConstraintEQ, name="Sum of u_i(\Beta) = 1", rhs=1)

    # we assume that each marginal value function has the same number of breakpoints
    if num_breakpoints == 0:
        return model

    # add monotonicity constraints for breakpoints
    for _, criterion in criteria.iterrows():
        criterion_type = criterion["criterion_type"]
        criterion_name = criterion["criterion_name"]
        if criterion_type == "cost":
            previous_breakpoint = criterion["weight"]
            for i in range(num_breakpoints):
                breakpoint = LpVariable(name=f"{criterion_name}_breakpoint_{i+1}", lowBound=0)
                # for cost type, the marginal value function must be NON-INcreasing
                # as such, the value at the next break point must be <= the previous
                model += LpConstraint(previous_breakpoint-breakpoint, sense=LpConstraintGE, name=f"{criterion_name}_monotonicity_{i+1}")
                previous_breakpoint = breakpoint

        if criterion_type == "gain":
            previous_breakpoint = 0
            for i in range(num_breakpoints):
                breakpoint = LpVariable(name=f"{criterion_name}_breakpoint_{i+1}", lowBound=0)
                # for gain type, the function should be NON-DEcreasing
                model += LpConstraint(breakpoint-previous_breakpoint, sense=LpConstraintGE, name=f"{criterion_name}_monotonicity_{i+1}")
                previous_breakpoint = breakpoint
    return model


In [230]:
model = ordinal_regression_model(alternatives, criteria, [("A", "B", ">")], 2)
model

c:\Users\marci\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Ordinal_Regression:
MINIMIZE
1*over_A + 1*over_B + 1*over_C + 1*over_D + 1*over_E + 1*over_F + 1*over_G + 1*over_H + 1*over_I + 1*over_J + 1*under_A + 1*under_B + 1*under_C + 1*under_D + 1*under_E + 1*under_F + 1*under_G + 1*under_H + 1*under_I + 1*under_J + 0
SUBJECT TO
A>B: U(A) - U(B) - over_A + over_B + under_A - under_B >= 0

u_a_experience: u_a_experience = 0

u_a_qualities: u_a_qualities = 0

u_a_expectation: u_a_expectation = 0

Sum_of_u_i(\Beta)_=_1: u_b_expectation + u_b_experience + u_b_qualities = 1

experience_monotonicity_1: experience_breakpoint_1 >= 0

experience_monotonicity_2: - experience_breakpoint_1 + experience_breakpoint_2
 >= 0

qualities_monotonicity_1: qualities_breakpoint_1 >= 0

qualities_monotonicity_2: - qualities_breakpoint_1 + qualities_breakpoint_2
 >= 0

expectation_monotonicity_1: - expectation_breakpoint_1 >= -0.3

expectation_monotonicity_2: expectation_breakpoint_1
 - expectation_breakpoint_2 >= 0

VARIABLES
U(A) <= 1 Continuous
U(B) <= 1 Continuou

In [237]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"Objective function value: {model.objective.value()}")

status: 1, Optimal
Objective function value: 0.0


In [240]:
for var in model.variables():
    print(var.value())

1.0
0.0
0.3
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.3
0.5
0.2
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
